In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:
data = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df = pd.read_csv(data)


print(df)

            Date    Open    High     Low    Last   Close  \
0     2018-09-28  234.05  235.95  230.20  233.50  233.75   
1     2018-09-27  234.55  236.80  231.10  233.80  233.25   
2     2018-09-26  240.00  240.00  232.50  235.00  234.25   
3     2018-09-25  233.30  236.75  232.00  236.25  236.10   
4     2018-09-24  233.55  239.20  230.75  234.00  233.30   
...          ...     ...     ...     ...     ...     ...   
2030  2010-07-27  117.60  119.50  112.00  118.80  118.65   
2031  2010-07-26  120.10  121.00  117.10  117.10  117.60   
2032  2010-07-23  121.80  121.95  120.25  120.35  120.65   
2033  2010-07-22  120.30  122.00  120.25  120.75  120.90   
2034  2010-07-21  122.10  123.00  121.05  121.10  121.55   

      Total Trade Quantity  Turnover (Lacs)  
0                  3069914          7162.35  
1                  5082859         11859.95  
2                  2240909          5248.60  
3                  2349368          5503.90  
4                  3423509          7999.55  
...

In [ ]:
df = df.dropna()
print(df)

In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
print(df)

           Date    Open    High     Low    Last   Close  Total Trade Quantity  \
2034 2010-07-21  122.10  123.00  121.05  121.10  121.55                658666   
2033 2010-07-22  120.30  122.00  120.25  120.75  120.90                293312   
2032 2010-07-23  121.80  121.95  120.25  120.35  120.65                281312   
2031 2010-07-26  120.10  121.00  117.10  117.10  117.60                658440   
2030 2010-07-27  117.60  119.50  112.00  118.80  118.65                586100   
...         ...     ...     ...     ...     ...     ...                   ...   
4    2018-09-24  233.55  239.20  230.75  234.00  233.30               3423509   
3    2018-09-25  233.30  236.75  232.00  236.25  236.10               2349368   
2    2018-09-26  240.00  240.00  232.50  235.00  234.25               2240909   
1    2018-09-27  234.55  236.80  231.10  233.80  233.25               5082859   
0    2018-09-28  234.05  235.95  230.20  233.50  233.75               3069914   

      Turnover (Lacs)  
203

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [9]:
train_data = scaled_data[:int(0.8*len(df))]
test_data = scaled_data[int(0.8*len(df)):]

In [10]:
def create_dataset(data, time_step):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

In [16]:
X_train, Y_train = create_dataset(train_data, 10)
X_test, Y_test = create_dataset(test_data, 10)

In [17]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [18]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [19]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [21]:
model.fit(X_train, Y_train)

51/51 [==============================] - 1s 16ms/step - loss: 3.9923e-04


In [22]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

13/13 [==============================] - 0s 5ms/step


In [23]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
Y_train = scaler.inverse_transform([Y_train])
Y_test = scaler.inverse_transform([Y_test])

In [24]:
train_rmse = np.sqrt(np.mean(np.power((Y_train - train_predict),2)))
test_rmse = np.sqrt(np.mean(np.power((Y_test - test_predict),2)))

In [25]:
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

Train RMSE: 31.830463754719947
Test RMSE: 74.57875149480343
